<a href="https://colab.research.google.com/github/a2m-dotcom/DLBCL_Pub/blob/main/CellViTSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!sudo apt-get install libvips openslide gcc g++ libopencv-core-dev libopencv-imgproc-dev libsnappy-dev libgeos-dev llvm libjpeg-dev libpng-dev libtiff-dev


In [ ]:
pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Install system package
!apt-get update
!apt-get install -y openslide-tools

# Install Python bindings
!pip install openslide-python

In [ ]:
pip install cellvit

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda create -n cellviaplusplus python=3.10 -y

In [ ]:
!pip install cellvit

In [ ]:
!pip install torchaudio

In [ ]:
!pip install cupy-cuda12x

In [ ]:
import cupy as cp
x = cp.array([1, 2, 3])
print(x * 10)

In [ ]:
!nvidia-smi

In [ ]:
!pip install cucim

In [ ]:
!cellvit-check

In [ ]:
from google.colab import drive
drive.mount('/content/drivee')

In [ ]:
import subprocess
from pathlib import Path
import os

cache_file = "/root/.cache/cellvit/CellViT-256-x40-AMP.pth"
if os.path.exists(cache_file):
    print("⚠️ Removing corrupted model checkpoint...")
    os.remove(cache_file)
# Google Drive paths
wsi_dir = Path("/content/drive/MyDrive/DLBCLMORPH/DLBCL3")       # Folder containing .svs files
out_base_dir = wsi_dir / "Results"                        # "Results" subfolder
out_base_dir.mkdir(parents=True, exist_ok=True)

# List all SVS files
svs_files = list(wsi_dir.glob("*.svs"))

if not svs_files:
    print("❌ No SVS files found in", wsi_dir)
else:
    print(f"Found {len(svs_files)} SVS files.")

# Loop through each WSI
for wsi_file in svs_files:
    base_name = wsi_file.stem
    print(f"\n🔹 Processing {wsi_file} ...")

    slide_out_dir = out_base_dir / base_name
    slide_out_dir.mkdir(exist_ok=True)

    # CellViT command
    cmd = [
        "cellvit-inference",
        "--model", "HIPT",
        "--nuclei_taxonomy", "nucls_super",
        "--gpu", "0",
        "--outdir", str(slide_out_dir),
        "--geojson",
        "--ray_worker", "2",
        "process_wsi",
        "--wsi_path", str(wsi_file),
       "--wsi_mpp", "0.25",  # This is the approximate mpp for 30x
    "--wsi_magnification", "40"  # Changed from 40
    ]

    print("Running:", " ".join(cmd))

    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed on {wsi_file}")
        print("STDOUT:", e.stdout)
        print("STDERR:", e.stderr)
        continue

    # Rename output files so they match the input SVS name
    for file in slide_out_dir.glob("*"):
        if file.suffix in [".json", ".geojson"]:
            new_name = f"{base_name}{file.suffix}"
            file.rename(slide_out_dir / new_name)

    print(f"✅ Finished {wsi_file} → Results in {slide_out_dir}")